In [1]:
import tensorflow as tf
from tensorflow.python.keras.applications import vgg16
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

import os

C:\Users\ADMIN\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ADMIN\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ADMIN\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ADMIN\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

In [2]:
train_dir = 'face2/train'
validation_dir = 'face2/validation'
test_dir='face2/test'


train_jaydev_dir = os.path.join(train_dir, 'jaydev')
train_rohan_dir = os.path.join(train_dir, 'rohan')


validation_jaydev_dir = os.path.join(validation_dir, 'jaydev')
validation_rohan_dir = os.path.join(validation_dir, 'rohan')

test_jaydev_dir = os.path.join(test_dir, 'jaydev')
test_rohan_dir = os.path.join(test_dir, 'rohan')

In [3]:
train_jaydev_dir_fnames = os.listdir( train_jaydev_dir)


print(train_jaydev_dir_fnames[:10])

['1 (2).jpg', '1 (3).jpg', '1 (4).jpg', '1 (5).jpg', '1 (6).jpg', '1 (7).jpg', '1.jpg', '10 (2).jpg', '10 (3).jpg', '10 (4).jpg']


In [4]:
model = tf.keras.models.Sequential([
    # input shape is the desired size of the image 150x150 with 3 bytes color
    #  first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    #  second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #  fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #  fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten 
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('jaydev') and 1 for the other ('rohan')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
validation_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    #batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  validation_datagen.flow_from_directory(validation_dir,
                                                         #batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))

Found 1355 images belonging to 2 classes.
Found 431 images belonging to 2 classes.


In [7]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              
                              epochs=10,
                             
                              verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
43/43 [==============================] - 36s 840ms/step - loss: 0.3700 - acc: 0.8236 - val_loss: 0.0521 - val_acc: 0.9977
Epoch 2/10
43/43 [==============================] - 21s 489ms/step - loss: 0.0089 - acc: 0.9993 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 3/10
43/43 [==============================] - 21s 491ms/step - loss: 0.0010 - acc: 1.0000 - val_loss: 9.8391e-04 - val_acc: 1.0000
Epoch 4/10
43/43 [==============================] - 21s 485ms/step - loss: 6.9151e-04 - acc: 1.0000 - val_loss: 5.4452e-06 - val_acc: 1.0000
Epoch 5/10
43/43 [==============================] - 21s 478ms/step - loss: 0.0022 - acc: 1.0000 - val_loss: 3.3666e-04 - val_acc: 1.0000
Epoch 6/10
43/43 [==============================] - 21s 484ms/step - loss: 7.0938e-05 - acc: 1.0000 - val_loss: 3.1825e-06 - val_acc: 1.0000
Epoch 7/10
43/43 [==============================] - 21s 477ms/step - loss: 2.7772e-05 - acc: 1.0000 - val_loss: 9.3524e-06 - val_a

In [8]:
import numpy as np
from keras.preprocessing import image

import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0)

while(True):
    return_value, image = camera.read()
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        face=image[y:y+h,x:x+w]
        face=cv2.resize(face,(150,150))
        #cv2.imwrite('try/try'+str(i)+'.jpg', face)# here specify ur folder
        image=cv2.rectangle(image, (x,y), (x+w,y+h), (255,0,0), 3)
        
    
    #path='try/try'+str(i)+'.jpg' 
    #img=cv2.imread(path)
  
        z=np.array(face)
        z=np.expand_dims(z, axis=0)
        images = np.vstack([z])

        classes = model.predict(images, batch_size=10)

        print(classes[0])

        if classes[0]>0:
            image=cv2.putText(image,'rohan',(x+60,y-10),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),2)
            print(" is a rohan")

        else:
            image=cv2.putText(image,'jaydev',(x+60,y-10),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),2)
            print(" is a jaydev")
        
    cv2.imshow('photu',image)
    if cv2.waitKey(1) & 0xFF == ord('q'): # when 'q' is pressed, vdo recording stops #heyy here arg in writekey specifies- 
                                        # -motion time , less is the number vdo will run fast ,as you increase the number-
                                        # -vdo will go slow
        break

del(camera)
cv2.waitKey(0)
cv2.destroyAllWindows()



 
  # predicting images


Using TensorFlow backend.


[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a

[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[0

[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.345231e-38]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a jaydev
[0.]
 is a

[1.4636227e-05]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[0.]
 is a jaydev
[1.]
 is a rohan
[1.]
 is a rohan
[1.]
 is a rohan
[0.]

In [38]:
import numpy as np


from keras.preprocessing import image

 


img=cv2.imread('face/test/rohan/59.jpg')
img=cv2.resize(img,(150,150))
  
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
  
classes = model.predict(images, batch_size=10)
  
print(classes[0])
  
if classes[0]>0:
    print(" is a rohan")

else:
    print(" is a jaydev")

[1.]
 is a rohan
